In [23]:
# Sciences
import face_recognition
import numpy as np

# Scraping
import praw, psaw
import requests
from io import BytesIO
from datetime import datetime

# Other
import pickle
from tqdm import tqdm
from PIL import Image
import traceback
import os

In [24]:
all_encodings = np.load('SCUT-encodings.npy')
labels = np.load('SCUT-labels.npy')

In [25]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, make_scorer

network = MLPRegressor(validation_fraction = 0, solver='adam', max_iter= 1000).fit(all_encodings, labels)
scores = cross_val_score(network, all_encodings, labels, scoring=make_scorer(mean_absolute_error))
print("Avg Mean Absolute Error: {0}, Std: {1}".format(scores.mean(), scores.std()))

Avg Mean Absolute Error: 0.2994605338167279, Std: 0.003406495515663741


In [29]:
pic_path = "encodings/pics_1425-050720.pkl"
selfie_path = "encodings/selfies_1647-050720.pkl"
roast_path = "encodings/roastme_1901-050820.pkl"
pic_encodings, pic_metadata = np.load(pic_path, allow_pickle=True)
selfie_encodings, selfie_metadata = np.load(selfie_path, allow_pickle=True)
roast_encodings, roast_metadata = np.load(roast_path, allow_pickle=True)
print(pic_encodings.shape)
print(selfie_encodings.shape)
print(roast_encodings.shape)

(33082, 128)
(10560, 128)
(75612, 128)


In [36]:
def runAndOut(encoding, fileName):
    rating = network.predict(encoding)
    print("Mean rating (1-5): {:0.2f}, std: {:1.1f}".format(rating.mean(), rating.std()))
    
    outfile = open(fileName,'wb')
    pickle.dump(rating,outfile)
    outfile.close()
    

In [40]:
runAndOut(pic_encodings, "ratings/pic_ratings.pkl")

Mean rating (1-5): 2.77, std: 0.6


In [41]:
runAndOut(selfie_encodings, "ratings/selfie_ratings.pkl")

Mean rating (1-5): 3.22, std: 0.6


In [42]:
runAndOut(roast_encodings, "ratings/roast_ratings.pkl")

Mean rating (1-5): 2.97, std: 0.5
